<h1>Intro to Data Science by Blueprint</h1>

In [1]:
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import statsmodels.api as sm # statsmodels for linear regression

from sklearn import preprocessing # data preprocessing

In [2]:
df = pd.read_csv('data/2016.csv')

In [3]:
df.head()

,Country,Region,Happiness Rank,Happiness Score,Lower Confidence Interval,Upper Confidence Interval,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Denmark,Western Europe,1,7.526,7.460,7.592,1.44178,1.16374,0.79504,0.57941,0.44453,0.36171,2.73939
1,Switzerland,Western Europe,2,7.509,7.428,7.590,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2.69463
2,Iceland,Western Europe,3,7.501,7.333,7.669,1.42666,1.18326,0.86733,0.56624,0.14975,0.47678,2.83137
3,Norway,Western Europe,4,7.498,7.421,7.575,1.57744,1.12690,0.79579,0.59609,0.35776,0.37895,2.66465
4,Finland,Western Europe,5,7.413,7.351,7.475,1.40598,1.13464,0.81091,0.57104,0.41004,0.25492,2.82596


In [4]:
X = df.iloc[:,6:]
y = df['Happiness Score']

In [5]:
scaler = preprocessing.MinMaxScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)
y = scaler.fit_transform(y.reshape(-1, 1))

In [ ]:
results = sm.OLS(y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.971
Model:                            OLS   Adj. R-squared:                  0.970
Method:                 Least Squares   F-statistic:                     728.3
Date:                Sat, 01 Jul 2017   Prob (F-statistic):          2.33e-112
Time:                        22:50:25   Log-Likelihood:                 139.15
No. Observations:                 157   AIC:                            -264.3
Df Residuals:                     150   BIC:                            -242.9
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=================================================================================================
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Economy (GDP per Capita)          0.3385      0.076      4.479      0.000       0.189       0.488
Family                            0.0995      0.050      1.980      0.050       0.000       0.199
Health (Life Expectancy)          0.1663      0.063      2.620      0.010       0.041       0.292
Freedom                           0.0659      0.045      1.470      0.144      -0.023       0.154
Trust (Government Corruption)     0.1385      0.045      3.107      0.002       0.050       0.227
Generosity                       -0.0728      0.054     -1.357      0.177      -0.179       0.033
Dystopia Residual                 0.3218      0.038      8.411      0.000       0.246       0.397
==============================================================================
Omnibus:                        5.215   Durbin-Watson:                   0.381
Prob(Omnibus):                  0.074   Jarque-Bera (JB):                2.958
Skew:                           0.091   Prob(JB):                        0.228
Kurtosis:                       2.353   Cond. No.                         16.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<h2>Important Features</h2>

Economy GDP per Capita, Health Life Expectancy, Dystopia Residual are main indicators in determining happiness

In [ ]:
plt.scatter(df['Economy (GDP per Capita)'], y)
plt.scatter(df['Health (Life Expectancy)'], y)
plt.scatter(df['Dystopia Residual'], y)
fig = plt.Figure(figsize=(8,6))
plt.show()